# Introducing: HistFrames
HistFrames are similar to data frames, but each row corresponds to a bin, like in a histogram. They do not aim to be fully featured histogram objects like you might find in other library, however. Jack of all trades; master of... also all trades.

Firstly, we need to define a binning

In [ ]:
import pyNUISANCE as pn

pn.log.set_level("Binning",pn.log.level.trace)

binning = pn.Binning.lin_space(50,0,50)
binning(1)

Then we can instantiate an empty HistFrame with this binning

In [ ]:
hf = pn.HistFrame(binning, "MCPrediction")
print(hf) #the printer only prints the first 20 rows by default

We can then add an entry to a bin

In [ ]:
import pyProSelecta as pps

evs = pn.EventSource("dune_argon_sf_10mega.nuwro.pb.gz")
if not evs:
    print("Error: failed to open input file")

def enu_GeV(ev):
    bpart = pps.sel.Beam(ev,14)
    print(bpart)
    if bpart:
        print(bpart.momentum().e() * 1E-3)
        return bpart.momentum().e() * 1E-3
    return -0

hf.reset() # reset it so that repeated running of this cell doesn't keep filling the histo
ev, cvw = evs.first()
enu = enu_GeV(ev)
hf.fill(enu, cvw) # for correctly normalizable distributions that work for different generators 
                  # we *always* need to keep track of the weights, even if they're all '1' for 
                  # our testing generator
print(hf)

Lets make a more sensibly binned histogram and fill it with a few events

In [ ]:
from math import log
binning = pn.Binning.lin_space(20,0,6, "E_nu [GeV]")
hf = pn.HistFrame(binning) #the default first column name is "mc"
for i, (ev, cvw) in enumerate(evs):
    print(cvw)
    hf.fill(enu_GeV(ev),cvw)
    if i >= 5:
        break

print(hf)

We can interrogate some of the binning metadata like below. The binning information is stored by bin index, this enables fully generic ndimensional binnings to be defined but can make them confusing to work with if you're use to information being stored by axis.

In [ ]:
print("Bin extents: %s" % hf.binning.bins)

In [ ]:
xaxbcs = [ x[0] for x in pn.Binning.get_bin_centers(hf.binning.bins) ]
print("Bin centers: %s" % xaxbcs)

Lets make a plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.errorbar(xaxbcs,
             hf.contents[:,0],yerr=np.sqrt(hf.variance[:,0]))
plt.show()